In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
import pickle
import time
import datetime

In [2]:
# Import qubit states Zero (|0>) and One (|1>), and Pauli operators (X, Y, Z)
from qiskit.opflow import Zero, One
from qiskit import QuantumCircuit, QuantumRegister, IBMQ, execute, transpile, Aer
from qiskit.tools.monitor import job_monitor
from qiskit.circuit import Parameter
from qiskit.transpiler.passes import RemoveBarriers

In [3]:
# Import QREM package
from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.ignis.mitigation import expectation_value

/var/folders/vr/d82vcp297v78vk4sb0_nzzdc0000gn/T/ipykernel_54868/3056705909.py:2: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter


In [4]:
# Import mitiq for zne
# unused for this file
import mitiq

In [5]:
# Import state tomography modules
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity

In [6]:
import sys
import importlib
sys.path.append("../../solutions/utils/")
import circuit_utils, zne_utils, tomography_utils
importlib.reload(circuit_utils)
importlib.reload(zne_utils) # unused for this file
importlib.reload(tomography_utils) # unused for this file
from circuit_utils import *
from zne_utils import zne_wrapper, zne_decoder
from tomography_utils import expvals_to_valid_rho

In [7]:
from qiskit.test.mock import FakeJakarta
backend = FakeJakarta()
# backend = Aer.get_backend("qasm_simulator")

In [8]:
num_qubits = 3

# The final time of the state evolution
target_time = np.pi

# Parameterize variable t to be evaluated at t=pi later
dt = Parameter('t')

# Convert custom quantum circuit into a gate
trot_gate = trotter_gate(dt)

# initial layout
initial_layout = [5,3,1]

# Number of trotter steps
num_steps_list = [1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100]
print("trotter step list: ", num_steps_list)

scale_factors = [1.0, 2.0, 3.0] # unused for this file

shots = 1 << 13
reps = 8 # unused

target_state = (One^One^Zero).to_matrix() # DO NOT CHANGE!!!

trotter step list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]


In [9]:
# QREM
qr = QuantumRegister(num_qubits, name="calq")
meas_calibs, state_labels = complete_meas_cal(qr=qr, circlabel='mcal')
# we have to feed initial_layout to calibration matrix
cal_job = execute(meas_calibs, backend=backend, shots=shots, optimization_level=3, initial_layout = initial_layout)
print('Job ID', cal_job.job_id())

retrieved_cal_job = cal_job
cal_results = retrieved_cal_job.result()
meas_fitter = CompleteMeasFitter(cal_results, state_labels, circlabel='mcal')

Job ID 3c9e4b7e-d156-4f18-8b0c-78adb8aae5d6


In [ ]:
zne_fid_list = []
zne_stddev_list = []

for num_steps in num_steps_list:
    
    print("trotter steps: ", num_steps)
    t1 = time.perf_counter()
    
    # Initialize quantum circuit for 3 qubits
    qr = QuantumRegister(num_qubits, name="q")
    qc = QuantumCircuit(qr)

    # Prepare initial state (remember we are only evolving 3 of the 7 qubits on jakarta qubits (q_5, q_3, q_1) corresponding to the state |110>)
    make_initial_state(qc, "110")  # DO NOT MODIFY (|q_5,q_3,q_1> = |110>)

    subspace_encoder_init110(qc, targets=[0, 1, 2]) # encode
    trotterize(qc, trot_gate, num_steps, targets=[1, 2]) # Simulate time evolution under H_heis3 Hamiltonian
    subspace_decoder(qc, targets=[0, 1, 2]) # decode

    # Evaluate simulation at target_time (t=pi) meaning each trotter step evolves pi/trotter_steps in time
    qc = qc.bind_parameters({dt: target_time / num_steps})
    print("created qc")

    # Generate state tomography circuits to evaluate fidelity of simulation
    st_qcs = state_tomography_circuits(qc, [0, 1, 2][::-1])  #! state tomography requires === BIG ENDIAN === 
    print("created st_qcs (length:", len(st_qcs), ")")

    # remove barriers
    st_qcs = [RemoveBarriers()(qc) for qc in st_qcs]
    print("removed barriers from st_qcs")

    # optimize circuit
    t3_st_qcs = transpile(st_qcs, optimization_level=3, basis_gates=["sx", "cx", "rz"])
    t3_st_qcs = transpile(t3_st_qcs, optimization_level=3, basis_gates=["sx", "cx", "rz"])
    print("created t3_st_qcs (length:", len(t3_st_qcs), ")")

    # zne wrapping
    zne_qcs = t3_st_qcs
    zne_qcs = zne_wrapper(t3_st_qcs, scale_factors = scale_factors, pt = True)
    print("created zne_qcs (length:", len(zne_qcs), ")")

    # optimization_level must be 0
    # feed initial_layout here to see the picture of the circuits before casting the job
    t3_zne_qcs = transpile(zne_qcs, optimization_level=0, basis_gates=["sx", "cx", "rz"], initial_layout=initial_layout)
    print("created t3_zne_qcs (length:", len(t3_zne_qcs), ")")
    
    jobs = []
    for _ in range(reps):
        #! CHECK: run t3_zne_qcs, with optimization_level = 0 and straightforward initial_layout
        job = execute(t3_zne_qcs, backend, shots=shots, optimization_level=0)
        print('Job ID', job.job_id())
        jobs.append(job)
    
    retrieved_jobs = jobs
    zne_fids = []
    for job in retrieved_jobs:
        raw_results = job.result()
        mit_results = meas_fitter.filter.apply(raw_results)
        zne_expvals = zne_decoder(num_qubits, mit_results, scale_factors = scale_factors)
        zne_rho = expvals_to_valid_rho(num_qubits, zne_expvals)
        zne_fid = state_fidelity(zne_rho, target_state)
        zne_fids.append(zne_fid)

    zne_fid_list.append(np.mean(zne_fids))
    zne_stddev_list.append(np.std(zne_fids))
    
    t2 = time.perf_counter()
    print('zne pt state tomography fidelity = {:.4f} \u00B1 {:.4f}'.format(np.mean(zne_fids), np.std(zne_fids)))
    print("time:", t2 - t1)
    print()

trotter steps:  1
created qc
created st_qcs (length: 27 )
removed barriers from st_qcs
created t3_st_qcs (length: 27 )
created zne_qcs (length: 81 )
created t3_zne_qcs (length: 81 )
Job ID 0b81da94-1b01-484f-978b-2ac70ebc1882
Job ID 19fbc871-06d4-464d-bea4-7f9ceab0b0fe
Job ID 3b4da250-ee1c-46c8-8f18-125f46727668
Job ID 0f93b0a5-bbc8-4095-bcbb-9aa10a0dc702
Job ID 17568ba8-e980-4499-b44d-e804dc97f34f
Job ID 3d6ce249-d028-4d6f-abe2-ff1744857c93
Job ID 075ad194-b396-40dc-b7dc-be1d0c282d0a
Job ID 254521a0-47cb-425c-85fb-043818c32b11
zne pt state tomography fidelity = 0.9844 ± 0.0046
time: 46.156308199

trotter steps:  2
created qc
created st_qcs (length: 27 )
removed barriers from st_qcs
created t3_st_qcs (length: 27 )


In [ ]:
with open("e2d1_zne_pt.pkl", "wb") as f:
    pickle.dump({"num_steps_list": num_steps_list, "fid": zne_fid_list, "stddev": zne_stddev_list}, f)

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table

In [ ]:
plt.plot(num_steps_list, zne_fid_list)